# Module2
- C++ example
  - Required: opencv, cmake
    ```shell
    sudo apt install libopencv-dev cmake
    ```
- Gray scale with brightness change

In [ ]:
import subprocess
import os
import cv2
import ipywidgets as widgets
from IPython.display import display, Image, clear_output

def build_module2(output_widget, cpp_dir):
    clean_module2(cpp_dir)
    current_dir = os.getcwd()
    try:
        os.chdir(cpp_dir)
        os.mkdir("build")
        os.chdir("build")
        subprocess.run(["cmake", ".."], check=True)
        subprocess.run(["make"], check=True)
        os.chdir("..")
    finally:
        os.chdir(current_dir)
    with output_widget:
        clear_output(wait=True)
        print(f"[Module2] Complete Build")

def run_module2(output_widget, cpp_dir:str, input_path:str, output_path:str, brightness:int):
    current_dir = os.getcwd()
    try:
        os.chdir(cpp_dir)
        os.chdir("build")
        subprocess.run(["./main", input_path, output_path, str(brightness)], check=True)
        os.chdir("..")
    finally:
        os.chdir(current_dir)
    with output_widget:
        clear_output(wait=True)
        print(f"[Module2] Complete Run: Brightness={brightness}")

def clean_module2(cpp_dir):
    current_dir = os.getcwd()
    try:
        os.chdir(cpp_dir)
        subprocess.run(["rm", "-rf", "build"], check=True)
    finally:
        os.chdir(current_dir)

def show_result(output_widget, file_path):
    _, ext = os.path.splitext(file_path)
    img = cv2.imread(file_path)
    _, buf = cv2.imencode(ext, img)

    with output_widget:
        clear_output(wait=True)
        print(f"[Module2] {file_path}")
        display(Image(data=buf.tobytes(), width=400))

def update_default_output_path(change):
    slider_value = change["new"]
    current_dir = os.getcwd()
    default_output_path = os.path.join(current_dir, f"data/Lenna_{slider_value}.png")
    form.children[1].value = default_output_path

def create_form():
    current_dir = os.getcwd()
    cpp_dir = os.path.join(current_dir, "cpp")

    # create slider
    slider = widgets.IntSlider(min=0, max=255, step=1, description='Brightness')
    # observe slider value change
    slider.observe(update_default_output_path, names="value")

    # create text field
    text1 = widgets.Text(description="Input Path:", 
                         value=os.path.join(current_dir, "data/Lenna.png"))
    text2 = widgets.Text(description="Output Path:", 
                         value=os.path.join(current_dir, f"data/Lenna_{slider.value}.png"))
    text1.layout.width = "600px"
    text2.layout.width = "600px"

    # create output area
    output_widget1 = widgets.Output()
    output_widget2 = widgets.Output()
    output_widget3 = widgets.Output()

    # create button1
    button1 = widgets.Button(description="Build Module2")
    button1.on_click(lambda _: build_module2(output_widget1, cpp_dir))

    # create button2
    button2 = widgets.Button(description="Run Module2")
    button2.on_click(lambda _: run_module2(output_widget2, cpp_dir, text1.value, text2.value, slider.value))

    # create button3
    button3 = widgets.Button(description="Show Result")
    button3.on_click(lambda _: show_result(output_widget3, text2.value))

    # create button4
    button4 = widgets.Button(description="Clean Module2")
    button4.on_click(lambda _: clean_module2(cpp_dir))

    form_items = [
        text1,
        text2,
        slider,
        button1,
        button2,
        button3,
        button4,
        output_widget1,
        output_widget2,
        output_widget3
    ]
    form = widgets.VBox(form_items, layout=widgets.Layout(width="800px"))

    return form

form = create_form()
display(form)